In [2]:
import numpy as np
import pandas as pd
import os
import plotly.express as px

In [3]:
# Import Definitions for Exception Codes and Performance Status Codes
os.chdir(r"Insert Data Folder Directory Here")
dimexc = pd.read_excel('vDimException.xlsx', converters={'ExceptionCode':str})
dimstatus = pd.read_excel('vDimStatus.xlsx', converters={'PerformStatusCode':str})
#also import exceptions_data.csv

In [ ]:
#Selects the Top Tracking Numbers from TException that have at least "4" exception codes, then merges it with TperformanceData
query = """
SELECT TOP 500000 * FROM [Track].[TException] a
INNER JOIN( SELECT TrackingNumber
FROM [Track].[TException] 
GROUP BY [TrackingNumber] 
HAVING COUNT(TrackingNumber) > 4) b
ON a.TrackingNumber = b.TrackingNumber

INNER JOIN( SELECT * 
FROM [Track].[TPerformanceData]
WHERE TNTStatusCode NOT IN ('00,07')) c
ON c.TrackingNumber = a.TrackingNumber
"""

In [ ]:
#merge dataframes with definitions
df = pd.merge(df, dimexc, left_on = 'ExcResultCode', right_on = 'ExceptionCode')
df = pd.merge(df, dimstatus, left_on = 'TNTStatusCode', right_on = 'PerformStatusCode')
df.shape
#The time it took me to make these merges seamless...

# Exploratory Data Analysis

In [ ]:
df['ExcDT'] = pd.to_datetime(df['ExcDT']) #Convert all dates to datetime objects
# least_recent_date = df['ExcDT'].min()
# recent_date = df['ExcDT'].max()
print("This data timeframe ranges from "+str(df['ExcDT'].min())+" to "+str(df['ExcDT'].max()))

In [ ]:
#Create IsLate column for binary classification
# 1 means the package is late 0 means its on time
df['IsLate'] = np.where(df['TNTStatusCode'].isin(['03','05','06','08']), 1, 0) 

In [ ]:
df = df.loc[:,~df.columns.duplicated()] # remove duplicate columns from dataframe
df = df.fillna(value = 'none') #Fill missing values

In [ ]:
#Filtering the dataframe in order to analyze the top occuring exception codes
topexc = df.groupby("ExceptionDesc").filter(lambda x: len(x) > 3000)
print(len(topexc)/len(df))
topexc.shape

In [ ]:
#Sunburst charts FTW
fig = px.sunburst(topexc, path=['InducerDescription','TNTStatusCode', 'ExcResultCode'],
                  hover_data=['ExceptionDesc','PerformDesc'])
fig.show()

## Prepare Dataframe for Label Encoding

In [ ]:
df2 = df.groupby('TrackingNumber').apply(lambda x: x['ExcReasonCode'].unique()) #create array with all exception codes grouped for a given tracking number
df2 = pd.DataFrame({'TrackingNumber':df2.index, 'ExceptionCodes':df2.values.tolist()}) #turn into dataframe
for i,row in df2.iterrows():
    row['ExceptionCodes'] = [x for x in row['ExceptionCodes'] if x != 'none'] #Remove null values

In [ ]:
dfx = pd.merge(df2, df.drop_duplicates(subset=['TrackingNumber']), on='TrackingNumber',how = 'left')

In [ ]:
#Dont remember what this cell does have fun haha, may have been a failure
q=0

def comp(x,q):
    i=0
    if x['ExcResultCode'] == 'none':
        inducer = 'X'
    else:
        try:
            inducer = defdict[x['ExcResultCode']]
        except:
            print(x['ExcResultCode'])
            q += 1
            pass
        finally:
            inducer = 'X'
    # print(x['ExceptionDesc']+' - Inducer: '+str(result))
    try:
        otherexc = set([defdict[i] for i in row['ExceptionCodes']])
    except:
        otherexc = []
    # other = set(otherexc)
    # print('other inducers:')
    # for p in otherexc: print(p)
    for x in otherexc: 
        if x != inducer: 
            i+=1
    # print(i)
    return inducer, i

dfx[['Inducer','test']]=dfx.apply(comp, axis=1, result_type="expand")
print(q)

# Testing a machine learning algorithm

In [ ]:
# df4 = pd.get_dummies(df2, columns=('ExceptionCodes'))
df3 = df2['ExceptionCodes'].str.join('|').str.get_dummies()
dfy = pd.concat([df2, df3], axis=1) 

In [ ]:
dfz = dfx[['TrackingNumber','IsLate']]

In [ ]:
dfy = pd.merge(dfy, dfz, on='TrackingNumber', how='left',)

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate
y = dfy.IsLate
X = dfy.drop(['TrackingNumber','IsLate','ExceptionCodes'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

print(confusion_matrix(rfc_pred,y_test))
print('Accuracy score:',accuracy_score(rfc_pred, y_test))
print(classification_report(rfc_pred, y_test))

cross_val_score_rfc = cross_validate(rfc, X_train, y_train,cv = 5,return_train_score=True)

print('Cross validation train_score',cross_val_score_rfc['train_score'].mean())
print('Cross validation test_score',cross_val_score_rfc['test_score'].mean())
winsound.Beep(440, 1000)

In [ ]:
#So what exception codes have the greatest predictive power?
feature_imp = rfc.feature_importances_.round(3)
df_feature_imp = pd.DataFrame({'Columns':pd.Series(X.columns),'Importance': feature_imp})
df_feature_imp = df_feature_imp.sort_values(by=['Importance'], ascending=False)
dff = df_feature_imp.nlargest(20,'Importance')

px.bar(dff,x='Columns',y='Importance',color='Importance')